In [1]:
# =========================================================================================
# 1. INSTALL DEPENDENCIES & IMPORTS
# =========================================================================================
!pip install -q timm

import os
import gc
import json
import random
import time
import argparse
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# =========================================================================================
# 2. CONFIGURATION
# =========================================================================================
class Config:
    # Model Selection
    model_name = 'vit_base_patch16_224'  # Options: vit_small_patch16_224, vit_base_patch16_224
    
    # Training Parameters
    img_size = 224
    batch_size = 32          # Adjust based on GPU memory (32 fits T4/P100 for ViT-Base)
    epochs = 20
    learning_rate = 1e-4     # 0.0001
    weight_decay = 0.01      # Regularization for ViT
    seed = 42
    num_workers = 2
    
    # Paths (Kaggle Standard Structure)
    root_dir = Path('/kaggle/input/vindr-spinexr-modified/vindr-spinexr-a-large-annotated-medical-image-dataset')
    train_img_dir = root_dir / 'train_png'  # Double check if folder is 'train_png' or 'train_images' in your dataset version
    test_img_dir = root_dir / 'test_png'    # Double check if folder is 'test_png' or 'test_images'
    train_csv = root_dir / 'annotations/train.csv'
    test_csv = root_dir / 'annotations/test.csv'
    
    # Output
    output_dir = Path('./outputs/vit_base_repro')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create output dir
Config.output_dir.mkdir(parents=True, exist_ok=True)

# Seeding for reproducibility
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(Config.seed)
print(f"Device: {Config.device}")

# =========================================================================================
# 3. DATASET CLASS
# =========================================================================================
class SpineDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
        # Pre-check existing files to avoid crashing during training
        self.valid_images = []
        missing = 0
        
        print(f"Checking image existence for {len(df)} entries...")
        for idx, row in df.iterrows():
            img_id = row['image_id']
            # Try png first, then maybe jpg/dicom depending on dataset (assuming PNG here based on your path)
            paths_to_try = [
                self.img_dir / f"{img_id}.png",
                self.img_dir / img_id
            ]
            
            found = False
            for p in paths_to_try:
                if p.exists():
                    self.valid_images.append((p, row['label']))
                    found = True
                    break
            
            if not found:
                missing += 1
                
        print(f"Verified {len(self.valid_images)} images. (Missing: {missing})")

    def __len__(self):
        return len(self.valid_images)

    def __getitem__(self, idx):
        img_path, label = self.valid_images[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return black image on failure to keep batch size consistent
            image = Image.new('RGB', (Config.img_size, Config.img_size), (0, 0, 0))
            
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.float32)

# =========================================================================================
# 4. DATA PROCESSING
# =========================================================================================
def process_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    
    # Logic: 
    # Group by image_id. 
    # If ANY lesion_type is NOT "No finding", label = 1 (Abnormal).
    # If ALL rows for an image are "No finding", label = 0 (Normal).
    
    # Helper to determine status
    def get_status(group):
        if (group['lesion_type'] == 'No finding').all():
            return 0 # Normal
        return 1 # Abnormal

    # Group and map
    image_labels = df.groupby('image_id').apply(get_status).reset_index()
    image_labels.columns = ['image_id', 'label']
    
    return image_labels

print("Processing Train Data...")
train_df_full = process_dataframe(Config.train_csv)

# Optional: Split validation from train csv if test.csv is strictly for final testing
# Assuming we use train.csv for train/val split
train_df, val_df = train_test_split(train_df_full, test_size=0.2, stratify=train_df_full['label'], random_state=Config.seed)

print(f"Train Set: {len(train_df)} | Validation Set: {len(val_df)}")
print(f"Train Class Distribution: \n{train_df['label'].value_counts()}")

# =========================================================================================
# 5. TRANSFORMS & LOADERS
# =========================================================================================
# ViT specific transforms (ImageNet normalization)
train_transforms = transforms.Compose([
    transforms.Resize((Config.img_size, Config.img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # ViT often uses 0.5 mean/std, or ImageNet
])

val_transforms = transforms.Compose([
    transforms.Resize((Config.img_size, Config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Create Datasets
train_dataset = SpineDataset(train_df, Config.train_img_dir, transform=train_transforms)
val_dataset = SpineDataset(val_df, Config.train_img_dir, transform=val_transforms) # Using train dir for val split

# Create Loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=Config.batch_size, 
    shuffle=True, 
    num_workers=Config.num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=Config.batch_size, 
    shuffle=False, 
    num_workers=Config.num_workers,
    pin_memory=True
)

# Calculate Class Weight
num_normal = (train_df['label'] == 0).sum()
num_abnormal = (train_df['label'] == 1).sum()
pos_weight = torch.tensor([num_normal / num_abnormal]).to(Config.device)
print(f"Using Positive Weight: {pos_weight.item():.4f}")

# =========================================================================================
# 6. MODEL SETUP
# =========================================================================================
print(f"Initializing {Config.model_name}...")
try:
    model = timm.create_model(Config.model_name, pretrained=True, num_classes=1)
    print("Loaded pretrained weights.")
except:
    print("Could not load pretrained weights. Check internet connection. Initializing random weights.")
    model = timm.create_model(Config.model_name, pretrained=False, num_classes=1)

model = model.to(Config.device)

# Optimization
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.AdamW(model.parameters(), lr=Config.learning_rate, weight_decay=Config.weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.epochs, eta_min=1e-6)

# =========================================================================================
# 7. TRAINING LOOP
# =========================================================================================
def train_one_epoch(epoch_index):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch_index}/{Config.epochs} [Train]", leave=False)
    
    for images, labels in pbar:
        images, labels = images.to(Config.device), labels.to(Config.device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze(1) # Ensure shape [batch] not [batch, 1]
        
        # Handle case where batch size is 1 (squeeze removes batch dim)
        if outputs.ndim == 0: outputs = outputs.unsqueeze(0)
            
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (preds == labels).sum().item()
        
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
    return running_loss / len(train_loader), correct / total

def validate_one_epoch(epoch_index):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    
    pbar = tqdm(val_loader, desc=f"Epoch {epoch_index}/{Config.epochs} [Val]", leave=False)
    
    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(Config.device), labels.to(Config.device)
            
            outputs = model(images).squeeze(1)
            if outputs.ndim == 0: outputs = outputs.unsqueeze(0)
                
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
    # Calculate Metrics
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    try:
        auroc = roc_auc_score(all_labels, all_probs)
    except:
        auroc = 0.5 # Fallback if only one class exists in batch
        
    return running_loss / len(val_loader), accuracy, auroc, all_labels, all_probs

# =========================================================================================
# 8. EXECUTION
# =========================================================================================
best_auroc = 0.0
history = []

print(f"\nStarting training for {Config.epochs} epochs...")

for epoch in range(1, Config.epochs + 1):
    # Train
    train_loss, train_acc = train_one_epoch(epoch)
    
    # Val
    val_loss, val_acc, val_auroc, _, _ = validate_one_epoch(epoch)
    
    # Scheduler
    scheduler.step()
    curr_lr = optimizer.param_groups[0]['lr']
    
    # Logging
    print(f"Epoch {epoch}: "
          f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | AUROC: {val_auroc:.4f} | LR: {curr_lr:.6f}")
    
    # Save checkpoint if best
    if val_auroc > best_auroc:
        best_auroc = val_auroc
        torch.save(model.state_dict(), Config.output_dir / "best_model.pth")
        print(f"--> New Best AUROC! Saved model.")
        
    # Save History
    history.append([epoch, train_loss, train_acc, val_loss, val_acc, val_auroc])

# =========================================================================================
# 9. FINAL EVALUATION & METRICS
# =========================================================================================
print("\nRunning Final Evaluation on Best Model...")

# Load best weights
model.load_state_dict(torch.load(Config.output_dir / "best_model.pth"))
model.eval()

# Re-run validation logic on validation set to get final metrics
_, final_acc, final_auroc, y_true, y_probs = validate_one_epoch("Final")
y_pred = (np.array(y_probs) > 0.5).astype(int)

# Metrics with Bootstrap CI
def bootstrap_metric(y_true, y_probs, metric_fn, n_boot=1000):
    scores = []
    rng = np.random.RandomState(Config.seed)
    indices = np.arange(len(y_true))
    
    for _ in range(n_boot):
        idx = rng.choice(indices, len(indices), replace=True)
        try:
            score = metric_fn(np.array(y_true)[idx], np.array(y_probs)[idx])
            scores.append(score)
        except:
            pass # Handle single-class batches
    
    return np.percentile(scores, 2.5), np.percentile(scores, 97.5)

# Calculate final metrics
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
f1 = f1_score(y_true, y_pred)

# Get CIs
auroc_ci = bootstrap_metric(y_true, y_probs, roc_auc_score)
# For F1, Sensitivity, Specificity, we need binary preds
# Note: Simplified CI calculation for brevity
sens_ci = (0.0, 0.0) # Placeholder or implement full bootstrap loop as needed

print("\n" + "="*50)
print("FINAL RESULTS - ViT Base Patch16 224")
print("="*50)
print(f"AUROC       : {final_auroc*100:.2f}% (CI: {auroc_ci[0]*100:.1f}-{auroc_ci[1]*100:.1f})")
print(f"F1 Score    : {f1*100:.2f}%")
print(f"Sensitivity : {sensitivity*100:.2f}%")
print(f"Specificity : {specificity*100:.2f}%")
print("-" * 50)
print("Confusion Matrix:")
print(f"TN: {tn} | FP: {fp}")
print(f"FN: {fn} | TP: {tp}")
print("="*50)

# Save history
hist_df = pd.DataFrame(history, columns=['Epoch', 'Train_Loss', 'Train_Acc', 'Val_Loss', 'Val_Acc', 'Val_AUROC'])
hist_df.to_csv(Config.output_dir / 'training_log.csv', index=False)
print("Saved training log.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda
Processing Train Data...
Train Set: 6711 | Validation Set: 1678
Train Class Distribution: 
label
0    3408
1    3303
Name: count, dtype: int64
Checking image existence for 6711 entries...
Verified 6527 images. (Missing: 184)
Checking image existence for 1678 entries...
Verified 1635 images. (Missing: 43)
Using Positive Weight: 1.0318
Initializing vit_base_patch16_224...


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loaded pretrained weights.

Starting training for 20 epochs...


...

Epoch 20/20 [Train]:   0%|          | 0/204 [00:00<?, ?it/s]

Epoch 20/20 [Val]:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch 20: Train Loss: 0.0858 | Acc: 0.9678 | Val Loss: 0.8233 | Val Acc: 0.7823 | AUROC: 0.8638 | LR: 0.000001

Running Final Evaluation on Best Model...


Epoch Final/20 [Val]:   0%|          | 0/52 [00:00<?, ?it/s]


FINAL RESULTS - ViT Base Patch16 224
AUROC       : 86.81% (CI: 85.0-88.3)
F1 Score    : 78.05%
Sensitivity : 78.09%
Specificity : 78.48%
--------------------------------------------------
Confusion Matrix:
TN: 649 | FP: 178
FN: 177 | TP: 631
Saved training log.


In [2]:
import os
import shutil
from IPython.display import FileLink

# ==========================================
# SAFEGUARD: SAVE & DOWNLOAD
# ==========================================

# 1. Define output paths
OUTPUT_DIR = Path('./outputs/vit_base_repro')
ARCHIVE_NAME = 'vit_model_results'

print(f"Compressing results from {OUTPUT_DIR}...")

# 2. Create a ZIP file of the output directory
# This ensures you get the model weights, logs, and prediction CSVs
shutil.make_archive(ARCHIVE_NAME, 'zip', OUTPUT_DIR)

print(f"Compression complete: {ARCHIVE_NAME}.zip")

# 3. Generate Download Link
# Click this link immediately after training finishes!
print(f"\n⬇️ CLICK HERE TO DOWNLOAD RESULTS ⬇️")
display(FileLink(f'{ARCHIVE_NAME}.zip'))

Compressing results from outputs/vit_base_repro...
Compression complete: vit_model_results.zip

⬇️ CLICK HERE TO DOWNLOAD RESULTS ⬇️


/kaggle/working/vit_model_results.zip